In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

<span style="font-size: 15px;">Neural network layers are the fundamental building blocks that transform input data into meaningful representations. Each layer type is designed for specific data structures and tasks. The **shape of the input and output tensors** plays a crucial role in understanding how these layers operate.

In what follows, we investigate the most common layer types in PyTorch:

1. **nn.Linear**: Fully connected layers for general-purpose transformations
2. **nn.Conv1d**: 1D convolutions for sequential/temporal data
3. **nn.Conv2d**: 2D convolutions for image data
4. **nn.Conv3d**: 3D convolutions for volumetric/video data

Throughout this notebook:
- $B$ denotes the batch size
- $C_{\text{in}}$ and $C_{\text{out}}$ denote input and output channels/features
- Spatial dimensions are denoted by $L$ (length), $H$ (height), $W$ (width), $D$ (depth)
</span>

**Overview**

| Layer | Input Shape | Output Shape | Primary Use Cases | Key Parameters |
|-------|-------------|--------------|-------------------|----------------|
| **nn.Linear** | $(*, H_{\text{in}})$ | $(*, H_{\text{out}})$ | MLPs, classification heads, dense connections, Transformer projections | `in_features`, `out_features`, `bias` |
| **nn.Conv1d** | $(B, C_{\text{in}}, L)$ | $(B, C_{\text{out}}, L_{\text{out}})$ | Time series, audio, text (1D sequences) | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding` |
| **nn.Conv2d** | $(B, C_{\text{in}}, H, W)$ | $(B, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ | Images, feature maps, 2D spatial data | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `dilation`, `groups` |
| **nn.Conv3d** | $(B, C_{\text{in}}, D, H, W)$ | $(B, C_{\text{out}}, D_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ | Videos, medical imaging (CT/MRI), 3D point clouds | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `dilation`, `groups` |


| Layer | Input Shape | Output Shape | Primary Use Cases | Key Parameters |
|-------|-------------|--------------|-------------------|----------------|
| **nn.ConvTranspose1d** | $(B, C_{\text{in}}, L)$ | $(B, C_{\text{out}}, L_{\text{out}})$ | Audio generation, sequence upsampling, WaveNet decoders | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `output_padding` |
| **nn.ConvTranspose2d** | $(B, C_{\text{in}}, H, W)$ | $(B, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ | Image generation (GANs), semantic segmentation decoders, autoencoders | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `output_padding` |
| **nn.ConvTranspose3d** | $(B, C_{\text{in}}, D, H, W)$ | $(B, C_{\text{out}}, D_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ | Video generation, 3D medical image reconstruction, volumetric upsampling | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `output_padding` |


Detailed explanations of each layer, including mathematical formulations and implementation examples, follow below.

## nn.Linear

<span style="font-size: 15px;">


### Where is it used?

The `nn.Linear` layer (also known as a fully connected or dense layer) applies a linear transformation to the incoming data. It is the most fundamental building block in neural networks and is used in:

- **Multi-Layer Perceptrons (MLPs)**: The backbone of simple feedforward networks
- **Classification heads**: Final layers that map features to class logits
- **Transformer architectures**: Q, K, V projections and feed-forward networks
- **Autoencoders**: Encoding and decoding dense representations
- **Regression tasks**: Mapping features to continuous outputs

### Input and Output Shapes

- **Input**: $(*, H_{\text{in}})$ where $*$ means any number of dimensions and $H_{\text{in}}$ is the number of input features
- **Output**: $(*, H_{\text{out}})$ where all dimensions except the last remain unchanged

**Important**: The linear transformation is applied to the **last dimension** only.

### Default Arguments

```python
nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
```

- `in_features` (int): Size of each input sample (required)
- `out_features` (int): Size of each output sample (required)
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- 
### Mathematical Formulation

The linear layer applies the following transformation:

$$
{y_{\rm out}}^{i,j,\cdots, k} = \omega_{k z} \, {x_{\rm in}}^{i,j,\cdots, z} + b^{k}
$$


where the indices "$i,j,\cdots$" represent an arbitrary dimension, i.e., $*$. This makes the layer **batch-agnostic**. As we see,
- $x$ is the input tensor of shape $(*, H_{\text{in}})$
- $\omega$ is the weight matrix of shape $(H_{\text{out}}, H_{\text{in}})$
- $b$ is the bias vector of shape $(H_{\text{out}})$
- $y$ is the output tensor of shape $(*, H_{\text{out}})$
- **Notice**, that, all elements in the arbitrary dimension "$i,j,\cdots$" get exactly the same biases and weights.




### Weight Initialization

Weights are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k = \frac{1}{H_{\text{in}}}$

</span>

In [27]:
# Basic nn.Linear usage
# Create a linear layer that maps 20 input features to 30 output features
linear = nn.Linear(20, 30)

# Check the weight and bias shapes
print(f"Weight shape: {linear.weight.shape}")  # (out_features, in_features)
print(f"Bias shape: {linear.bias.shape}")      # (out_features,)

Weight shape: torch.Size([30, 20])
Bias shape: torch.Size([30])


In [28]:
# Example 1: Simple 2D input (batch_size, in_features)
batch_size = 128
in_features = 20
out_features = 30

linear = nn.Linear(in_features, out_features)
models_weights = linear.weight
models_biases = linear.bias
x = torch.randn(batch_size, in_features)
output = linear(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

# Lets now try to reproduce the output manually:

output_manual = torch.einsum('bi,oi->bo', x, models_weights) + models_biases.unsqueeze(0).expand(batch_size, -1)
print(f'Outputs match: {torch.all(output_manual==output).item()}')

Input shape: torch.Size([128, 20])
Output shape: torch.Size([128, 30])
Outputs match: True


In [29]:
# Example 3: Without bias
linear_no_bias = nn.Linear(20, 30, bias=False)
print(f"Has bias: {linear_no_bias.bias is not None}")

# Verify the transformation manually
x = torch.randn(5, 20)
output_layer = linear_no_bias(x)
output_manual = x @ linear_no_bias.weight.T  # y = x @ W^T

print(f"Outputs match: {torch.allclose(output_layer, output_manual)}")

Has bias: False
Outputs match: True


## Convolutional layers

Before we start discussing the types of convolutional layers, we briefly discuss some of the important padding types appearing in these layers:



| Padding | Conv1d | Conv2d | Conv3d | Meaning |
|---------|--------|--------|--------|---------|
| `padding=0` | `0` | `(0,0)` | `(0,0,0)` | No padding, output shrinks |
| `padding='valid'` | `0` | `(0,0)` | `(0,0,0)` | Equivalent to `padding=0` |
| `padding='same'` | auto | auto | auto | Output size = Input size (requires `stride=1`) |
| `padding=k//2` | `k//2` | `(k//2, k//2)` | `(k//2, k//2, k//2)` | Preserves size for odd kernel $k$ with `stride=1`, `dilation=1` |

**Common recipe to preserve spatial dimensions:**

For `kernel_size=k`, `stride=1`, `dilation=1`:
```python
# These are equivalent:
padding = 'same'
padding = k // 2   # only works for odd k
```

**Examples preserving size:**

| Kernel | Padding |
|--------|---------|
| 3 | 1 |
| 5 | 2 |
| 7 | 3 |

**General formula:** `padding = (kernel_size - 1) // 2` for odd kernels with `stride=1`, `dilation=1`.

### nn.Conv1d

<span style="font-size: 15px;">

#### Where is it used?

The `nn.Conv1d` layer applies a 1D convolution over an input signal composed of several input planes. It is primarily used for:

- **Time series analysis**: Stock prices, sensor data, weather patterns
- **Audio processing**: Speech recognition, music classification
- **Text/NLP**: Character-level or word-level sequence modeling
- **Signal processing**: Any 1D sequential data

#### Input and Output Shapes

- **Input**: $(B, C_{\text{in}}, L_{\text{in}})$ where:
  - $B$ = batch size
  - $C_{\text{in}}$ = number of input channels
  - $L_{\text{in}}$ = length of the input sequence

- **Output**: $(B, C_{\text{out}}, L_{\text{out}})$ where:
  - $C_{\text{out}}$ = number of output channels (number of filters)
  - $L_{\text{out}} = \left\lfloor\frac{L_{\text{in}} + 2 \times \text{padding} - \text{dilation} \times (\text{kernel\_size} - 1) - 1}{\text{stride}} + 1\right\rfloor$

#### Default Arguments

```python
nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, 
          dilation=1, groups=1, bias=True, padding_mode='zeros')
```

- `in_channels` (int): Number of channels in the input (required)
- `out_channels` (int): Number of channels produced by the convolution (required)
- `kernel_size` (int or tuple): Size of the convolving kernel (required)
- `stride` (int): Stride of the convolution. Default: `1`
- `padding` (int or str): Zero-padding added to both sides. Default: `0`. Can be `'same'` or `'valid'`
- `dilation` (int): Spacing between kernel elements. Default: `1`
- `groups` (int): Number of blocked connections. Default: `1`
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- `padding_mode` (str): `'zeros'`, `'reflect'`, `'replicate'`, or `'circular'`. Default: `'zeros'`

#### Weight Shape

The weight tensor has shape: $(C_{\text{out}}, \frac{C_{\text{in}}}{\text{groups}}, \text{kernel\_size})$



#### Mathematical Formulation

For an input of size $(B, C_{\text{in}}, L)$ and output of size $(B, C_{\text{out}}, L_{\text{out}})$, the convolution from $x_{\rm in}[b, c, i]$ to $y_{\rm out}[b, c, i] $ is performed as follows:


* For `groups`$ = 1$:
$$
\begin{aligned}
y_{\rm out}[b, c_{out}, i] = \sum_{c_{\rm in}=0}^{C_{\rm in} -1} \sum_{k=0}^{K -1}~{\tilde x}_{\rm in}[b,~ c_{\rm in},~ s\cdot i+~d\cdot k] \omega[c_{\rm out}, c_{\rm in}, k] + \beta[c_{\rm out}]
\end{aligned}
$$

* For `groups`$ > 1$ and, `in_channels % groups`$~=~$ `out_channels % groups`$=0$:
$$
\begin{aligned}
y_{\rm out}[b, c_{out}, i] = \sum_{j=0}^{C_{\rm in}/g -1} \sum_{k=0}^{K -1}~{\tilde x}_{\rm in}[b,~ m(c_{\rm out})\cdot \frac{C_{\rm in}}{g} + j,~ s\cdot i+~d\cdot k] \omega[c_{\rm out}, j, k] + \beta[c_{\rm out}]
\end{aligned}
$$

where ${\tilde x}_{\rm in}$ is the padded input tensor, it is simply equal to $x_{\rm in}$ if padding is zero. Moreover, $g = $ `groups`, $s = $ `stride`, $d = $ `dilation` , and $m(c_{\rm out}) = {\rm floor}( c_{out} \cdot  g / C_{out} )$.



**Remark**: When `groups` $=1$, each filter combines information from all input channels. Otherwise, channels are processed in isolated groups, reducing computation and parameters but limiting cross-channel interaction.

</span>

In [30]:
# Basic nn.Conv1d usage
# Input: 16 channels, Output: 33 channels, kernel size: 3
conv1d = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3)

print(f"Weight shape: {conv1d.weight.shape}")  # (out_channels, in_channels, kernel_size)
print(f"Bias shape: {conv1d.bias.shape}")      # (out_channels,)

Weight shape: torch.Size([33, 16, 3])
Bias shape: torch.Size([33])


In [31]:
# Example 1: Basic convolution without padding
batch_size = 20
in_channels = 16
out_channels = 33
kernel_size = 3
L_in = 50  # Input sequence length

conv1d = nn.Conv1d(in_channels, out_channels, kernel_size)
x = torch.randn(batch_size, in_channels, L_in)
output = conv1d(x)

# Calculate expected output length: L_out = (L_in + 2*padding - dilation*(kernel_size-1) - 1) / stride + 1
L_out_expected = (L_in + 2*0 - 1*(kernel_size-1) - 1) // 1 + 1

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Expected L_out: {L_out_expected}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 48])
Expected L_out: 48


In [32]:
# Example 2: Convolution with padding to maintain spatial dimension
# For kernel_size=3, padding=1 maintains the length (with stride=1)
conv1d_same = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3, padding=1)
x = torch.randn(20, 16, 50)
output = conv1d_same(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Length preserved: {x.shape[2] == output.shape[2]}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 50])
Length preserved: True


In [33]:
# Example 3: Using padding='same' (requires stride=1)
conv1d_same_str = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=5, padding='same')
x = torch.randn(20, 16, 50)
output = conv1d_same_str(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 50])


In [38]:
# Example 4: Convolution with stride > 1 (downsampling)
conv1d_stride = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3, stride=2)
x = torch.randn(20, 16, 50)
output = conv1d_stride(x)

# L_out = (50 + 2*0 - 1*(3-1) - 1) // 2 + 1 = (50 - 2 - 1) // 2 + 1 = 47//2 + 1 = 23 + 1 = 24
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 24])


In [39]:
# Example 5: Dilated convolution (for larger receptive field)
conv1d_dilated = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3, dilation=2)
x = torch.randn(20, 16, 50)
output = conv1d_dilated(x)

# L_out = (50 + 2*0 - 2*(3-1) - 1) / 1 + 1 = (50 - 4 - 1) + 1 = 46
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Effective kernel size with dilation=2: {2*(3-1)+1}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 46])
Effective kernel size with dilation=2: 5


### nn.Conv2d

<span style="font-size: 15px;">

#### Where is it used?

The `nn.Conv2d` layer applies a 2D convolution over an input signal composed of several input planes. It is the backbone of computer vision and is used in:

- **Image classification**: CNNs for recognizing objects in images
- **Object detection**: YOLO, Faster R-CNN, etc.
- **Semantic segmentation**: U-Net, DeepLab, etc.
- **Image generation**: GANs, VAEs
- **Feature extraction**: Any 2D spatial data processing

#### Input and Output Shapes

- **Input**: $(B, C_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ where:
  - $B$ = batch size
  - $C_{\text{in}}$ = number of input channels (e.g., 3 for RGB images)
  - $H_{\text{in}}$ = height of the input
  - $W_{\text{in}}$ = width of the input

- **Output**: $(B, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ where:
  - $C_{\text{out}}$ = number of output channels (number of filters)
  - $H_{\text{out}} = \left\lfloor\frac{H_{\text{in}} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1\right\rfloor$
  - $W_{\text{out}} = \left\lfloor\frac{W_{\text{in}} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1\right\rfloor$

#### Default Arguments
```python
nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
          dilation=1, groups=1, bias=True, padding_mode='zeros')
```

- `in_channels` (int): Number of channels in the input (required)
- `out_channels` (int): Number of channels produced by the convolution (required)
- `kernel_size` (int or tuple): Size of the convolving kernel (required)
- `stride` (int or tuple): Stride of the convolution. Default: `1`
- `padding` (int, tuple, or str): Zero-padding added to both sides. Default: `0`. Can be `'same'` or `'valid'`
- `dilation` (int or tuple): Spacing between kernel elements. Default: `1`
- `groups` (int): Number of blocked connections. Default: `1`
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- `padding_mode` (str): `'zeros'`, `'reflect'`, `'replicate'`, or `'circular'`. Default: `'zeros'`

#### Weight Shape

The weight tensor has shape: $(C_{\text{out}}, \frac{C_{\text{in}}}{\text{groups}}, \text{kernel\_size}[0], \text{kernel\_size}[1])$

#### Mathematical Formulation

For an input of size $(B, C_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ and output of size $(B, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$, the convolution from $x_{\rm in}[b, c, h, w]$ to $y_{\rm out}[b, c, h, w]$ is performed as follows:

* For `groups`$ = 1$:
$$
\begin{aligned}
y_{\rm out}[b, c_{out}, h, w] = \sum_{c_{\rm in}=0}^{C_{\rm in} -1} \sum_{k_h=0}^{K_h -1} \sum_{k_w=0}^{K_w -1}~{\tilde x}_{\rm in}[b,~ c_{\rm in},~ s_h h+d_h k_h,~ s_w w + d_w k_w] \cdot \omega[c_{\rm out}, c_{\rm in}, k_h, k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

* For `groups`$ > 1$ and, `in_channels % groups`$~=~$ `out_channels % groups`$=0$:
$$
\begin{aligned}
y_{\rm out}[b, c_{out}, h, w] = \sum_{j=0}^{C_{\rm in}/g -1} \sum_{k_h=0}^{K_h -1} \sum_{k_w=0}^{K_w -1}~{\tilde x}_{\rm in}[b,~ m(c_{\rm out})\frac{C_{\rm in}}{g} + j,~ s_h h+d_h \cdot k_h,~ s_w \cdot w + d_w \cdot k_w] \cdot \omega[c_{\rm out}, j, k_h, k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

where ${\tilde x}_{\rm in}$ is the padded input tensor (equal to $x_{\rm in}$ if padding is zero). Moreover, $g = $ `groups`, $(s_h, s_w) = $ `stride`, $(d_h, d_w) = $ `dilation`, $(K_h, K_w) = $ `kernel_size`, and $m(c_{\rm out}) = {\rm floor}( c_{\rm out} \cdot g / C_{\rm out} )$.

**Remarks**:
1. When `groups` $=1$, each filter combines information from all input channels. Otherwise, channels are processed in isolated groups, reducing computation and parameters but limiting cross-channel interaction.
2. If `kernel_size = k` is just a number, then PyTorch automatically assumes that $(K_h, K_w) = (k, k)$.

</span>

In [43]:
# Basic nn.Conv2d usage
# Input: 16 channels, Output: 33 channels, kernel size: 3x3
conv2d = nn.Conv2d(in_channels=16, out_channels=33, kernel_size=3)

print(f"Weight shape: {conv2d.weight.shape}")  # (out_channels, in_channels, kH, kW)
print(f"Bias shape: {conv2d.bias.shape}")      # (out_channels,)

Weight shape: torch.Size([33, 16, 3, 3])
Bias shape: torch.Size([33])


In [44]:
# Example 1: Square kernels and equal stride
batch_size = 20
in_channels = 16
H, W = 50, 100

conv2d = nn.Conv2d(16, 33, kernel_size=3, stride=2)
x = torch.randn(batch_size, in_channels, H, W)
output = conv2d(x)

# H_out = (50 + 2*0 - 1*(3-1) - 1) / 2 + 1 = 24
# W_out = (100 + 2*0 - 1*(3-1) - 1) / 2 + 1 = 49
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50, 100])
Output shape: torch.Size([20, 33, 24, 49])


In [47]:
# Example 2: Non-square kernels and unequal stride with padding
conv2d = nn.Conv2d(16, 33, kernel_size=(3, 5), stride=(2, 1), padding=(4, 2))
x = torch.randn(20, 16, 50, 100)
output = conv2d(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50, 100])
Output shape: torch.Size([20, 33, 28, 100])


In [49]:
# Example 3: Maintaining spatial dimensions with appropriate padding
# For kernel_size=3 and stride=1, padding=1 preserves dimensions
conv2d_same = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
x = torch.randn(1, 3, 224, 224)  # Typical ImageNet input
output = conv2d_same(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Spatial dimensions preserved: {x.shape[2:] == output.shape[2:]}")

Input shape: torch.Size([1, 3, 224, 224])
Output shape: torch.Size([1, 64, 224, 224])
Spatial dimensions preserved: True


In [50]:
# Example 4: Dilated convolution
conv2d_dilated = nn.Conv2d(16, 33, kernel_size=(3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
x = torch.randn(20, 16, 50, 100)
output = conv2d_dilated(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50, 100])
Output shape: torch.Size([20, 33, 26, 100])


In [51]:
# Example 5: Depthwise convolution (groups = in_channels)
# Each input channel is convolved separately with its own filter
in_channels = 32
conv2d_depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels)

print(f"Depthwise conv weight shape: {conv2d_depthwise.weight.shape}")
# Note: each filter only sees 1 input channel (in_channels/groups = 32/32 = 1)

x = torch.randn(1, in_channels, 64, 64)
output = conv2d_depthwise(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Depthwise conv weight shape: torch.Size([32, 1, 3, 3])
Input shape: torch.Size([1, 32, 64, 64])
Output shape: torch.Size([1, 32, 64, 64])


In [52]:
# Understanding groups parameter more deeply
# Standard conv: each output channel sees ALL input channels
# groups=2: split into 2 parallel convs, each seeing half the input channels

# Standard convolution
conv_standard = nn.Conv2d(4, 8, kernel_size=3, groups=1)
print(f"Standard conv weight shape: {conv_standard.weight.shape}")
# Shape: (8, 4, 3, 3) - each of 8 filters sees all 4 input channels

# Grouped convolution with groups=2
conv_grouped = nn.Conv2d(4, 8, kernel_size=3, groups=2)
print(f"Grouped conv weight shape: {conv_grouped.weight.shape}")
# Shape: (8, 2, 3, 3) - each of 8 filters sees only 2 input channels (4/2)

# Depthwise convolution (groups = in_channels)
conv_depthwise = nn.Conv2d(4, 4, kernel_size=3, groups=4)
print(f"Depthwise conv weight shape: {conv_depthwise.weight.shape}")
# Shape: (4, 1, 3, 3) - each of 4 filters sees only 1 input channel

Standard conv weight shape: torch.Size([8, 4, 3, 3])
Grouped conv weight shape: torch.Size([8, 2, 3, 3])
Depthwise conv weight shape: torch.Size([4, 1, 3, 3])


In [54]:
# Verify the output size formula
def conv2d_output_size(H_in, W_in, kernel_size, stride=1, padding=0, dilation=1):
    """Calculate Conv2d output dimensions"""
    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)
    if isinstance(stride, int):
        stride = (stride, stride)
    if isinstance(padding, int):
        padding = (padding, padding)
    if isinstance(dilation, int):
        dilation = (dilation, dilation)
    
    H_out = (H_in + 2*padding[0] - dilation[0]*(kernel_size[0]-1) - 1) // stride[0] + 1
    W_out = (W_in + 2*padding[1] - dilation[1]*(kernel_size[1]-1) - 1) // stride[1] + 1
    return H_out, W_out

# Test with various configurations
configs = [
    {'kernel_size': 3, 'stride': 1, 'padding': 1},
    {'kernel_size': 3, 'stride': 2, 'padding': 1},
    {'kernel_size': 5, 'stride': 2, 'padding': 2},
    {'kernel_size': 3, 'stride': 1, 'padding': 1, 'dilation': 2},
]

H_in, W_in = 64, 64
x = torch.randn(1, 3, H_in, W_in)

for config in configs:
    conv = nn.Conv2d(3, 16, **config)
    output = conv(x)
    H_out_calc, W_out_calc = conv2d_output_size(H_in, W_in, **config)
    print(f"Config: {config}")
    print(f"  Calculated: ({H_out_calc}, {W_out_calc}), Actual: {output.shape[2:]}")
    print(f"  Match: {(H_out_calc, W_out_calc) == tuple(output.shape[2:])}")

Config: {'kernel_size': 3, 'stride': 1, 'padding': 1}
  Calculated: (64, 64), Actual: torch.Size([64, 64])
  Match: True
Config: {'kernel_size': 3, 'stride': 2, 'padding': 1}
  Calculated: (32, 32), Actual: torch.Size([32, 32])
  Match: True
Config: {'kernel_size': 5, 'stride': 2, 'padding': 2}
  Calculated: (32, 32), Actual: torch.Size([32, 32])
  Match: True
Config: {'kernel_size': 3, 'stride': 1, 'padding': 1, 'dilation': 2}
  Calculated: (62, 62), Actual: torch.Size([62, 62])
  Match: True


### Conv3d

<span style="font-size: 15px;">

#### Where is it used?

The `nn.Conv3d` layer applies a 3D convolution over an input signal composed of several input planes. It is used for:

- **Video analysis**: Action recognition, video classification
- **Medical imaging**: CT scans, MRI volumes, 3D ultrasound
- **3D object recognition**: Point cloud processing, voxel-based analysis
- **Spatiotemporal data**: Any data with 3 spatial/temporal dimensions

#### Input and Output Shapes

- **Input**: $(B, C_{\text{in}}, D_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ where:
  - $B$ = batch size
  - $C_{\text{in}}$ = number of input channels
  - $D_{\text{in}}$ = depth of the input (e.g., number of frames in video, slices in CT scan)
  - $H_{\text{in}}$ = height of the input
  - $W_{\text{in}}$ = width of the input

- **Output**: $(B, C_{\text{out}}, D_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ where:
  - $C_{\text{out}}$ = number of output channels (number of filters)
  - $D_{\text{out}} = \left\lfloor\frac{D_{\text{in}} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1\right\rfloor$
  - $H_{\text{out}} = \left\lfloor\frac{H_{\text{in}} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1\right\rfloor$
  - $W_{\text{out}} = \left\lfloor\frac{W_{\text{in}} + 2 \times \text{padding}[2] - \text{dilation}[2] \times (\text{kernel\_size}[2] - 1) - 1}{\text{stride}[2]} + 1\right\rfloor$

#### Default Arguments
```python
nn.Conv3d(in_channels, out_channels, kernel_size, stride=1, padding=0,
          dilation=1, groups=1, bias=True, padding_mode='zeros')
```

- `in_channels` (int): Number of channels in the input volume (required)
- `out_channels` (int): Number of channels produced by the convolution (required)
- `kernel_size` (int or tuple): Size of the convolving kernel (required)
- `stride` (int or tuple): Stride of the convolution. Default: `1`
- `padding` (int, tuple, or str): Zero-padding added to all sides. Default: `0`. Can be `'same'` or `'valid'`
- `dilation` (int or tuple): Spacing between kernel elements. Default: `1`
- `groups` (int): Number of blocked connections. Default: `1`
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- `padding_mode` (str): `'zeros'`, `'reflect'`, `'replicate'`, or `'circular'`. Default: `'zeros'`

#### Weight Shape

The weight tensor has shape: $(C_{\text{out}}, \frac{C_{\text{in}}}{\text{groups}}, \text{kernel\_size}[0], \text{kernel\_size}[1], \text{kernel\_size}[2])$

#### Mathematical Formulation

For an input of size $(B, C_{\text{in}}, D_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ and output of size $(B, C_{\text{out}}, D_{\text{out}}, H_{\text{out}}, W_{\text{out}})$, the convolution from $x_{\rm in}[b, c, d, h, w]$ to $y_{\rm out}[b, c, d, h, w]$ is performed as follows:

* For `groups`$ = 1$:
$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, d, h, w] = \sum_{c_{\rm in}=0}^{C_{\rm in} -1} \sum_{k_d=0}^{K_d -1} \sum_{k_h=0}^{K_h -1} \sum_{k_w=0}^{K_w -1}~{\tilde x}_{\rm in}[b,~ c_{\rm in},~ s_d \cdot d + d_d \cdot k_d,~ s_h \cdot h + d_h \cdot k_h,~ s_w \cdot w + d_w \cdot k_w] \cdot \omega[c_{\rm out}, c_{\rm in}, k_d, k_h, k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

* For `groups`$ > 1$ and, `in_channels % groups`$~=~$ `out_channels % groups`$=0$:
$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, d, h, w] = \sum_{j=0}^{C_{\rm in}/g -1} \sum_{k_d=0}^{K_d -1} \sum_{k_h=0}^{K_h -1} \sum_{k_w=0}^{K_w -1}~{\tilde x}_{\rm in}[b,~ m(c_{\rm out})\frac{C_{\rm in}}{g} + j,~ s_d \cdot d + d_d\cdot  k_d,~ s_h \cdot h + d_h \cdot k_h,~ s_w\cdot  w + d_w\cdot  k_w] \cdot \omega[c_{\rm out}, j, k_d, k_h, k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

where ${\tilde x}_{\rm in}$ is the padded input tensor (equal to $x_{\rm in}$ if padding is zero). Moreover, $g = $ `groups`, $(s_d, s_h, s_w) = $ `stride`, $(d_d, d_h, d_w) = $ `dilation`, $(K_d, K_h, K_w) = $ `kernel_size`, and $m(c_{\rm out}) = \lfloor c_{\rm out} \cdot g / C_{\rm out} \rfloor$.

**Remarks**:
1. When `groups` $=1$, each filter combines information from all input channels. Otherwise, channels are processed in isolated groups, reducing computation and parameters but limiting cross-channel interaction.
2. If `kernel_size = k` is just a number, then PyTorch automatically assumes that $(K_d, K_h, K_w) = (k, k, k)$.

</span>

In [55]:
# Basic nn.Conv3d usage
# Input: 16 channels, Output: 33 channels, kernel size: 3x3x3
conv3d = nn.Conv3d(in_channels=16, out_channels=33, kernel_size=3)

print(f"Weight shape: {conv3d.weight.shape}")  # (out_channels, in_channels, kD, kH, kW)
print(f"Bias shape: {conv3d.bias.shape}")      # (out_channels,)

Weight shape: torch.Size([33, 16, 3, 3, 3])
Bias shape: torch.Size([33])


In [58]:
# Example 1: Cubic kernels and equal stride
conv3d = nn.Conv3d(16, 33, kernel_size=3, stride=2)
x = torch.randn(20, 16, 10, 50, 100)  # (batch, channels, depth, height, width)
output = conv3d(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 10, 50, 100])
Output shape: torch.Size([20, 33, 4, 24, 49])


In [60]:
# Example 2: Non-cubic kernels and unequal stride with padding
# kernel_size=(depth, height, width), stride=(sD, sH, sW), padding=(pD, pH, pW)
conv3d = nn.Conv3d(16, 33, kernel_size=(3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
x = torch.randn(20, 16, 10, 50, 100)
output = conv3d(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 10, 50, 100])
Output shape: torch.Size([20, 33, 8, 50, 99])


In [61]:
# Example 3: Video processing (batch, channels, frames, height, width)
# A typical video input: 3 RGB channels, 16 frames, 112x112 resolution
batch_size = 4
channels = 3
frames = 16
height = width = 112

conv3d_video = nn.Conv3d(channels, 64, kernel_size=3, stride=1, padding=1)
video_input = torch.randn(batch_size, channels, frames, height, width)
output = conv3d_video(video_input)

print(f"Input shape (video): {video_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Dimensions preserved: {video_input.shape[2:] == output.shape[2:]}")

Input shape (video): torch.Size([4, 3, 16, 112, 112])
Output shape: torch.Size([4, 64, 16, 112, 112])
Dimensions preserved: True


In [63]:
# Example 4: Medical imaging (CT scan volume)
# Input: single channel (grayscale), 32 slices, 128x128 resolution
conv3d_medical = nn.Conv3d(1, 16, kernel_size=3, padding=1)
ct_scan = torch.randn(1, 1, 32, 128, 128)  # (batch, channel, slices, height, width)
output = conv3d_medical(ct_scan)

print(f"CT scan input shape: {ct_scan.shape}")
print(f"Feature map output shape: {output.shape}")

CT scan input shape: torch.Size([1, 1, 32, 128, 128])
Feature map output shape: torch.Size([1, 16, 32, 128, 128])


In [64]:
# Verify the output size formula for Conv3d
def conv3d_output_size(D_in, H_in, W_in, kernel_size, stride=1, padding=0, dilation=1):
    """Calculate Conv3d output dimensions"""
    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size, kernel_size)
    if isinstance(stride, int):
        stride = (stride, stride, stride)
    if isinstance(padding, int):
        padding = (padding, padding, padding)
    if isinstance(dilation, int):
        dilation = (dilation, dilation, dilation)
    
    D_out = (D_in + 2*padding[0] - dilation[0]*(kernel_size[0]-1) - 1) // stride[0] + 1
    H_out = (H_in + 2*padding[1] - dilation[1]*(kernel_size[1]-1) - 1) // stride[1] + 1
    W_out = (W_in + 2*padding[2] - dilation[2]*(kernel_size[2]-1) - 1) // stride[2] + 1
    return D_out, H_out, W_out

# Test
D_in, H_in, W_in = 16, 64, 64
x = torch.randn(1, 3, D_in, H_in, W_in)

configs = [
    {'kernel_size': 3, 'stride': 1, 'padding': 1},
    {'kernel_size': 3, 'stride': 2, 'padding': 1},
    {'kernel_size': (3, 5, 5), 'stride': (1, 2, 2), 'padding': (1, 2, 2)},
]

for config in configs:
    conv = nn.Conv3d(3, 16, **config)
    output = conv(x)
    D_out_calc, H_out_calc, W_out_calc = conv3d_output_size(D_in, H_in, W_in, **config)
    print(f"Config: {config}")
    print(f"  Calculated: ({D_out_calc}, {H_out_calc}, {W_out_calc}), Actual: {tuple(output.shape[2:])}")
    print(f"  Match: {(D_out_calc, H_out_calc, W_out_calc) == tuple(output.shape[2:])}")
    print()

Config: {'kernel_size': 3, 'stride': 1, 'padding': 1}
  Calculated: (16, 64, 64), Actual: (16, 64, 64)
  Match: True

Config: {'kernel_size': 3, 'stride': 2, 'padding': 1}
  Calculated: (8, 32, 32), Actual: (8, 32, 32)
  Match: True

Config: {'kernel_size': (3, 5, 5), 'stride': (1, 2, 2), 'padding': (1, 2, 2)}
  Calculated: (16, 32, 32), Actual: (16, 32, 32)
  Match: True



### Summary

In [66]:
# Summary comparison of all layers
print("=" * 80)
print("LAYER COMPARISON SUMMARY")
print("=" * 80)

# nn.Linear
linear = nn.Linear(512, 256)
x_linear = torch.randn(32, 512)
print(f"\nnn.Linear(512, 256):")
print(f"  Input shape:  {tuple(x_linear.shape)} -> (batch, in_features)")
print(f"  Output shape: {tuple(linear(x_linear).shape)} -> (batch, out_features)")
print(f"  Weight shape: {tuple(linear.weight.shape)} -> (out_features, in_features)")
print(f"  Parameters:   {sum(p.numel() for p in linear.parameters()):,}")

# nn.Conv1d
conv1d = nn.Conv1d(64, 128, kernel_size=3, padding=1)
x_conv1d = torch.randn(32, 64, 100)
print(f"\nnn.Conv1d(64, 128, kernel_size=3, padding=1):")
print(f"  Input shape:  {tuple(x_conv1d.shape)} -> (batch, in_channels, length)")
print(f"  Output shape: {tuple(conv1d(x_conv1d).shape)} -> (batch, out_channels, length)")
print(f"  Weight shape: {tuple(conv1d.weight.shape)} -> (out_ch, in_ch, kernel)")
print(f"  Parameters:   {sum(p.numel() for p in conv1d.parameters()):,}")

# nn.Conv2d
conv2d = nn.Conv2d(64, 128, kernel_size=3, padding=1)
x_conv2d = torch.randn(32, 64, 56, 56)
print(f"\nnn.Conv2d(64, 128, kernel_size=3, padding=1):")
print(f"  Input shape:  {tuple(x_conv2d.shape)} -> (batch, in_channels, H, W)")
print(f"  Output shape: {tuple(conv2d(x_conv2d).shape)} -> (batch, out_channels, H, W)")
print(f"  Weight shape: {tuple(conv2d.weight.shape)} -> (out_ch, in_ch, kH, kW)")
print(f"  Parameters:   {sum(p.numel() for p in conv2d.parameters()):,}")

# nn.Conv3d
conv3d = nn.Conv3d(64, 128, kernel_size=3, padding=1)
x_conv3d = torch.randn(4, 64, 16, 56, 56)
print(f"\nnn.Conv3d(64, 128, kernel_size=3, padding=1):")
print(f"  Input shape:  {tuple(x_conv3d.shape)} -> (batch, in_channels, D, H, W)")
print(f"  Output shape: {tuple(conv3d(x_conv3d).shape)} -> (batch, out_channels, D, H, W)")
print(f"  Weight shape: {tuple(conv3d.weight.shape)} -> (out_ch, in_ch, kD, kH, kW)")
print(f"  Parameters:   {sum(p.numel() for p in conv3d.parameters()):,}")

print("\n" + "=" * 80)

LAYER COMPARISON SUMMARY

nn.Linear(512, 256):
  Input shape:  (32, 512) -> (batch, in_features)
  Output shape: (32, 256) -> (batch, out_features)
  Weight shape: (256, 512) -> (out_features, in_features)
  Parameters:   131,328

nn.Conv1d(64, 128, kernel_size=3, padding=1):
  Input shape:  (32, 64, 100) -> (batch, in_channels, length)
  Output shape: (32, 128, 100) -> (batch, out_channels, length)
  Weight shape: (128, 64, 3) -> (out_ch, in_ch, kernel)
  Parameters:   24,704

nn.Conv2d(64, 128, kernel_size=3, padding=1):
  Input shape:  (32, 64, 56, 56) -> (batch, in_channels, H, W)
  Output shape: (32, 128, 56, 56) -> (batch, out_channels, H, W)
  Weight shape: (128, 64, 3, 3) -> (out_ch, in_ch, kH, kW)
  Parameters:   73,856

nn.Conv3d(64, 128, kernel_size=3, padding=1):
  Input shape:  (4, 64, 16, 56, 56) -> (batch, in_channels, D, H, W)
  Output shape: (4, 128, 16, 56, 56) -> (batch, out_channels, D, H, W)
  Weight shape: (128, 64, 3, 3, 3) -> (out_ch, in_ch, kD, kH, kW)
  Parame

In [67]:
# Key formulas for output size calculation
print("KEY FORMULAS FOR OUTPUT SIZE CALCULATION")
print("="*60)
print()
print("nn.Linear:")
print("  Output: (*, out_features)")
print("  The transformation is applied to the LAST dimension only")
print()
print("nn.Conv1d/Conv2d/Conv3d:")
print("  For each spatial dimension:")
print("  L_out = floor((L_in + 2*padding - dilation*(kernel_size-1) - 1) / stride + 1)")
print()
print("Special cases:")
print("  - padding='same' with stride=1: output size = input size")
print("  - padding=k//2 with odd kernel k and stride=1: preserves size")
print("  - stride=2 with appropriate padding: halves spatial dimensions")

KEY FORMULAS FOR OUTPUT SIZE CALCULATION

nn.Linear:
  Output: (*, out_features)
  The transformation is applied to the LAST dimension only

nn.Conv1d/Conv2d/Conv3d:
  For each spatial dimension:
  L_out = floor((L_in + 2*padding - dilation*(kernel_size-1) - 1) / stride + 1)

Special cases:
  - padding='same' with stride=1: output size = input size
  - padding=k//2 with odd kernel k and stride=1: preserves size
  - stride=2 with appropriate padding: halves spatial dimensions


## Transposed Convolutional Layers

<span style="font-size: 15px;">Transposed convolutional layers (often called "deconvolutions" or "fractionally strided convolutions") are essential building blocks for **upsampling** in neural networks. They perform the **inverse spatial transformation** of regular convolutions, where convolutions typically reduce spatial dimensions, transposed convolutions increase them.

**Why Transposed Convolutions?**



Regular convolutions with `stride > 1` **downsample** spatial dimensions. For many applications (image generation, segmentation, autoencoders), we need to **upsample**, to go from a smaller spatial representation back to a larger one.

**Key insight**: A transposed convolution is not literally the inverse of a convolution. Instead, it is the **transpose of the convolution operation viewed as a matrix multiplication**. If a convolution maps from a space of dimension $N$ to a space of dimension $M$, the transposed convolution maps from $M$ back to $N$.

Concretely:
- **Convolution** ($\text{stride}=2$): Input $L_{\text{in}} \to$ Output $L_{\text{out}} \approx L_{\text{in}}/2$ (downsampling)
- **Transposed Convolution** ($\text{stride}=2$): Input $L_{\text{in}} \to$ Output $L_{\text{out}} \approx 2 \cdot L_{\text{in}}$ (upsampling)

The `output_padding` parameter resolves the ambiguity that arises because multiple input sizes can map to the same output size under convolution.

In what follows, we investigate the transposed convolutional layer types in PyTorch:

1. **nn.ConvTranspose1d**: 1D transposed convolutions for sequential/temporal upsampling
2. **nn.ConvTranspose2d**: 2D transposed convolutions for image upsampling
3. **nn.ConvTranspose3d**: 3D transposed convolutions for volumetric/video upsampling

Throughout this notebook:
- $B$ denotes the batch size
- $C_{\text{in}}$ and $C_{\text{out}}$ denote input and output channels/features
- Spatial dimensions are denoted by $L$ (length), $H$ (height), $W$ (width), $D$ (depth)
</span>



Before we start discussing the types of transposed convolutional layers, we briefly discuss the key parameters:

| Parameter | Description |
|-----------|-------------|
| `stride` | Controls the **upsampling factor**. Unlike in `Conv`, larger stride means **larger** output. |
| `padding` | Reduces the output size (removes elements from the output). |
| `output_padding` | Adds extra elements to one side of the output to resolve size ambiguity. Must be `< stride`. |
| `dilation` | Spacing between kernel elements (same as in regular convolution). |

**Output size formula** (for each spatial dimension):

$$
L_{\text{out}} = (L_{\text{in}} - 1) \times \text{stride} - 2 \times \text{padding} + \text{dilation} \times (\text{kernel\_size} - 1) + \text{output\_padding} + 1
$$

**Common recipe for exact $2\times$ upsampling** (doubling spatial dimensions):

```python
# For exact 2x upsampling:
nn.ConvTranspose2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1)
# or
nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2, padding=0)

### nn.ConvTranspose1d


#### Where is it used?

The `nn.ConvTranspose1d` layer applies a 1D transposed convolution over an input signal composed of several input planes. It is primarily used for:

- **Audio generation**: WaveNet decoders, audio super-resolution
- **Sequence upsampling**: Increasing temporal resolution of time series
- **1D autoencoders**: Decoder networks for sequential data
- **Signal reconstruction**: Upsampling compressed signal representations

#### Input and Output Shapes

- **Input**: $(B, C_{\text{in}}, L_{\text{in}})$ where:
  - $B$ = batch size
  - $C_{\text{in}}$ = number of input channels
  - $L_{\text{in}}$ = length of the input sequence

- **Output**: $(B, C_{\text{out}}, L_{\text{out}})$ where:
  - $C_{\text{out}}$ = number of output channels
  - $L_{\text{out}} = (L_{\text{in}} - 1) \times \text{stride} - 2 \times \text{padding} + \text{dilation} \times (\text{kernel\_size} - 1) + \text{output\_padding} + 1$

#### Default Arguments

```python
nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride=1, padding=0,
                   output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
```

- `in_channels` (int): Number of channels in the input (required)
- `out_channels` (int): Number of channels produced by the convolution (required)
- `kernel_size` (int or tuple): Size of the convolving kernel (required)
- `stride` (int): Stride of the convolution (controls upsampling factor). Default: `1`
- `padding` (int): Zero-padding added to both sides of the input. **Reduces** output size. Default: `0`
- `output_padding` (int): Additional size added to one side of the output. Must be `< stride`. Default: `0`
- `dilation` (int): Spacing between kernel elements. Default: `1`
- `groups` (int): Number of blocked connections. Default: `1`
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- `padding_mode` (str): Only `'zeros'` is supported. Default: `'zeros'`

#### Weight Shape

The weight tensor has shape: $(C_{\text{in}}, \frac{C_{\text{out}}}{\text{groups}}, \text{kernel\_size})$

**Note**: The weight shape is **transposed** compared to `nn.Conv1d` which has shape $(C_{\text{out}}, \frac{C_{\text{in}}}{\text{groups}}, K)$.

#### Mathematical Formulation

For an input of size $(B, C_{\text{in}}, L_{\text{in}})$ and output of size $(B, C_{\text{out}}, L_{\text{out}})$, the transposed convolution from $x_{\rm in}[b, c, i]$ to $y_{\rm out}[b, c, j]$ can be understood as follows:

The transposed convolution is defined as the **gradient of a convolution with respect to its input**. Equivalently, it can be viewed as:

1. **Insert zeros** between input elements (determined by `stride`)
2. **Pad** the expanded input
3. Apply a **regular convolution** with the spatially flipped kernel

* For `groups`$ = 1$:
$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, j] = \sum_{c_{\rm in}=0}^{C_{\rm in}-1} \sum_{k=0}^{K-1} \tilde{x}_{\rm in}[b, c_{\rm in}, j + p - d \cdot k] \cdot \omega[c_{\rm in}, c_{\rm out}, K - 1 - k] + \beta[c_{\rm out}]
\end{aligned}
$$

where $\tilde{x}_{\rm in}$ is the input after zero-insertion (inserting $s-1$ zeros between each element) and appropriate padding adjustment.

Alternatively, the operation can be expressed as a **summation over input positions** that contribute to each output position:

$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, j] = \sum_{c_{\rm in}=0}^{C_{\rm in}-1} \sum_{i=0}^{L_{\rm in}-1} \sum_{k=0}^{K-1} \mathbf{1}_{[j = s \cdot i + d \cdot k - p + p_{\rm out}]} \cdot x_{\rm in}[b, c_{\rm in}, i] \cdot \omega[c_{\rm in}, c_{\rm out}, k] + \beta[c_{\rm out}]
\end{aligned}
$$

where $s = $ `stride`, $d = $ `dilation`, $p = $ `padding`, $p_{\rm out} = $ `output_padding`, and $\mathbf{1}_{[\cdot]}$ is the indicator function.

* For `groups`$ > 1$, the channels are split into groups analogously to regular convolutions.

**Remark**: The key intuition is that each input element "broadcasts" its value to multiple output positions through the kernel, with the kernel flipped spatially.

</span>

In [68]:
# Basic nn.ConvTranspose1d usage
# Input: 16 channels, Output: 33 channels, kernel size: 3
conv_transpose1d = nn.ConvTranspose1d(in_channels=16, out_channels=33, kernel_size=3)

print(f"Weight shape: {conv_transpose1d.weight.shape}")  # (in_channels, out_channels, kernel_size)
print(f"Bias shape: {conv_transpose1d.bias.shape}")      # (out_channels,)

Weight shape: torch.Size([16, 33, 3])
Bias shape: torch.Size([33])


In [76]:
# Example 1: Basic transposed convolution (upsampling)
batch_size = 20
in_channels = 16
out_channels = 33
kernel_size = 3
L_in = 50  # Input sequence length

conv_transpose1d = nn.ConvTranspose1d(in_channels, out_channels, kernel_size)
x = torch.randn(batch_size, in_channels, L_in)
output = conv_transpose1d(x)

# Calculate expected output length: L_out = (L_in - 1) * stride - 2*padding + dilation*(kernel_size-1) + output_padding + 1
# With defaults: stride=1, padding=0, dilation=1, output_padding=0
L_out_expected = (L_in - 1) * 1 - 2*0 + 1*(kernel_size-1) + 0 + 1
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Expected L_out: {L_out_expected}")
print(f"Note: Output is LARGER than input: {L_out_expected > x.shape[2]}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 52])
Expected L_out: 52
Note: Output is LARGER than input: True


In [77]:
# Example 2: 2x upsampling with stride=2
conv_transpose1d_upsample = nn.ConvTranspose1d(in_channels=16, out_channels=33, kernel_size=4, stride=2, padding=1)
x = torch.randn(20, 16, 50)
output = conv_transpose1d_upsample(x)

# L_out = (50-1)*2 - 2*1 + 1*(4-1) + 0 + 1 = 98 - 2 + 3 + 1 = 100
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Upsampling factor: {output.shape[2] / x.shape[2]}x")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 100])
Upsampling factor: 2.0x


In [78]:
# Example 3: Using output_padding to resolve ambiguity
# When stride > 1, multiple input sizes can produce the same output size after Conv1d
# output_padding helps specify the exact output size we want

conv_transpose1d_op = nn.ConvTranspose1d(in_channels=16, out_channels=33, kernel_size=3, stride=2, padding=1, output_padding=1)
x = torch.randn(20, 16, 50)
output = conv_transpose1d_op(x)

# L_out = (50-1)*2 - 2*1 + 1*(3-1) + 1 + 1 = 98 - 2 + 2 + 1 + 1 = 100
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 100])


In [79]:
# Example 4: Comparison with Conv1d (inverse relationship)
# ConvTranspose1d can reverse the spatial transformation of Conv1d

L_original = 100
x_original = torch.randn(1, 16, L_original)

# Downsample with Conv1d
conv1d_down = nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1)
x_down = conv1d_down(x_original)

# Upsample with ConvTranspose1d (matching parameters)
conv_transpose1d_up = nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1)
x_up = conv_transpose1d_up(x_down)

print(f"Original shape: {x_original.shape}")
print(f"After Conv1d (downsample): {x_down.shape}")
print(f"After ConvTranspose1d (upsample): {x_up.shape}")
print(f"Spatial dimension restored: {x_original.shape[2] == x_up.shape[2]}")

Original shape: torch.Size([1, 16, 100])
After Conv1d (downsample): torch.Size([1, 32, 50])
After ConvTranspose1d (upsample): torch.Size([1, 16, 100])
Spatial dimension restored: True


In [80]:
# Example 5: Dilated transposed convolution
conv_transpose1d_dilated = nn.ConvTranspose1d(in_channels=16, out_channels=33, kernel_size=3, dilation=2)
x = torch.randn(20, 16, 50)
output = conv_transpose1d_dilated(x)

# L_out = (50-1)*1 - 2*0 + 2*(3-1) + 0 + 1 = 49 + 4 + 1 = 54
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Effective kernel size with dilation=2: {2*(3-1)+1}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 54])
Effective kernel size with dilation=2: 5


In [82]:
# Verify the output size formula for ConvTranspose1d
def conv_transpose1d_output_size(L_in, kernel_size, stride=1, padding=0, dilation=1, output_padding=0):
    """Calculate ConvTranspose1d output length"""
    L_out = (L_in - 1) * stride - 2 * padding + dilation * (kernel_size - 1) + output_padding + 1
    return L_out

# Test with various configurations
L_in = 50
x = torch.randn(1, 16, L_in)

configs = [
    {'kernel_size': 3, 'stride': 1, 'padding': 0},
    {'kernel_size': 4, 'stride': 2, 'padding': 1},
    {'kernel_size': 3, 'stride': 2, 'padding': 1, 'output_padding': 1},
    {'kernel_size': 3, 'stride': 1, 'padding': 0, 'dilation': 2},
]

for config in configs:
    conv_t = nn.ConvTranspose1d(16, 33, **config)
    output = conv_t(x)
    L_out_calc = conv_transpose1d_output_size(L_in, **config)
    print(f"Config: {config}")
    print(f"  Calculated: {L_out_calc}, Actual: {output.shape[2]}")
    print(f"  Match: {L_out_calc == output.shape[2]}")

Config: {'kernel_size': 3, 'stride': 1, 'padding': 0}
  Calculated: 52, Actual: 52
  Match: True
Config: {'kernel_size': 4, 'stride': 2, 'padding': 1}
  Calculated: 100, Actual: 100
  Match: True
Config: {'kernel_size': 3, 'stride': 2, 'padding': 1, 'output_padding': 1}
  Calculated: 100, Actual: 100
  Match: True
Config: {'kernel_size': 3, 'stride': 1, 'padding': 0, 'dilation': 2}
  Calculated: 54, Actual: 54
  Match: True


### nn.ConvTranspose2d

<span style="font-size: 15px;">

#### Where is it used?

The `nn.ConvTranspose2d` layer applies a 2D transposed convolution over an input image composed of several input planes. It is the backbone of image upsampling and is used in:

- **Generative Adversarial Networks (GANs)**: Generator networks that upsample from latent vectors to images
- **Semantic segmentation**: Decoder networks in U-Net, FCN, etc.
- **Image super-resolution**: Upscaling low-resolution images
- **Autoencoders**: Decoder part for image reconstruction
- **Style transfer**: Reconstructing styled images at original resolution

#### Input and Output Shapes

- **Input**: $(B, C_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ where:
  - $B$ = batch size
  - $C_{\text{in}}$ = number of input channels
  - $H_{\text{in}}$ = height of the input
  - $W_{\text{in}}$ = width of the input

- **Output**: $(B, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ where:
  - $C_{\text{out}}$ = number of output channels
  - $H_{\text{out}} = (H_{\text{in}} - 1) \times \text{stride}[0] - 2 \times \text{padding}[0] + \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) + \text{output\_padding}[0] + 1$
  - $W_{\text{out}} = (W_{\text{in}} - 1) \times \text{stride}[1] - 2 \times \text{padding}[1] + \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) + \text{output\_padding}[1] + 1$

#### Default Arguments
```python
nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
                   output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
```

- `in_channels` (int): Number of channels in the input image (required)
- `out_channels` (int): Number of channels produced by the convolution (required)
- `kernel_size` (int or tuple): Size of the convolving kernel (required)
- `stride` (int or tuple): Stride of the convolution (controls upsampling factor). Default: `1`
- `padding` (int or tuple): Zero-padding added to both sides. **Reduces** output size. Default: `0`
- `output_padding` (int or tuple): Additional size added to one side of output. Must be `< stride`. Default: `0`
- `dilation` (int or tuple): Spacing between kernel elements. Default: `1`
- `groups` (int): Number of blocked connections. Default: `1`
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- `padding_mode` (str): Only `'zeros'` is supported. Default: `'zeros'`

#### Weight Shape

The weight tensor has shape: $(C_{\text{in}}, \frac{C_{\text{out}}}{\text{groups}}, \text{kernel\_size}[0], \text{kernel\_size}[1])$

**Note**: The weight shape is **transposed** compared to `nn.Conv2d` which has shape $(C_{\text{out}}, \frac{C_{\text{in}}}{\text{groups}}, K_h, K_w)$.

#### Mathematical Formulation

For an input of size $(B, C_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ and output of size $(B, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$, the transposed convolution from $x_{\rm in}[b, c, h, w]$ to $y_{\rm out}[b, c, h', w']$ is performed as follows:

* For `groups`$ = 1$:
$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, h', w'] = \sum_{c_{\rm in}=0}^{C_{\rm in}-1} \sum_{k_h=0}^{K_h-1} \sum_{k_w=0}^{K_w-1} \tilde{x}_{\rm in}[b, c_{\rm in}, h' + p_h - d_h k_h, w' + p_w - d_w k_w] \cdot \omega[c_{\rm in}, c_{\rm out}, K_h - 1 - k_h, K_w - 1 - k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

where $\tilde{x}_{\rm in}$ is the input after zero-insertion (inserting $s_h - 1$ and $s_w - 1$ zeros between elements along height and width respectively).

Equivalently, the operation can be expressed as a **summation over input positions**:

$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, h', w'] = \sum_{c_{\rm in}=0}^{C_{\rm in}-1} \sum_{h=0}^{H_{\rm in}-1} \sum_{w=0}^{W_{\rm in}-1} \sum_{k_h=0}^{K_h-1} \sum_{k_w=0}^{K_w-1} \mathbf{1}_{\substack{[h' = s_h h + d_h k_h - p_h + p_{\rm out,h}] \\ [w' = s_w w + d_w k_w - p_w + p_{\rm out,w}]}} \cdot x_{\rm in}[b, c_{\rm in}, h, w] \cdot \omega[c_{\rm in}, c_{\rm out}, k_h, k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

where $(s_h, s_w) = $ `stride`, $(d_h, d_w) = $ `dilation`, $(p_h, p_w) = $ `padding`, $(p_{\rm out,h}, p_{\rm out,w}) = $ `output_padding`, $(K_h, K_w) = $ `kernel_size`, and $\mathbf{1}_{[\cdot]}$ is the indicator function.

* For `groups`$ > 1$, the channels are split into groups analogously to regular convolutions.

**Remarks**:
1. Each input pixel "scatters" its influence to a $K_h \times K_w$ region in the output, weighted by the kernel.
2. If `kernel_size = k` is just a number, then PyTorch automatically assumes $(K_h, K_w) = (k, k)$.
3. The `output_padding` only adds to one side (not both), which is why it must be strictly less than `stride`.

</span>

In [83]:
# Basic nn.ConvTranspose2d usage
# Input: 16 channels, Output: 33 channels, kernel size: 3x3
conv_transpose2d = nn.ConvTranspose2d(in_channels=16, out_channels=33, kernel_size=3)

print(f"Weight shape: {conv_transpose2d.weight.shape}")  # (in_channels, out_channels, kH, kW)
print(f"Bias shape: {conv_transpose2d.bias.shape}")      # (out_channels,)

Weight shape: torch.Size([16, 33, 3, 3])
Bias shape: torch.Size([33])


In [85]:
# Example 1: Basic transposed convolution (slight upsampling)
batch_size = 20
in_channels = 16
H, W = 50, 100

conv_transpose2d = nn.ConvTranspose2d(16, 33, kernel_size=3)
x = torch.randn(batch_size, in_channels, H, W)
output = conv_transpose2d(x)

# H_out = (50-1)*1 - 2*0 + 1*(3-1) + 0 + 1 = 52
# W_out = (100-1)*1 - 2*0 + 1*(3-1) + 0 + 1 = 102
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50, 100])
Output shape: torch.Size([20, 33, 52, 102])


In [86]:
# Example 2: 2x upsampling (common in GANs and decoders)
# kernel_size=4, stride=2, padding=1 gives exact 2x upsampling
conv_transpose2d_2x = nn.ConvTranspose2d(16, 33, kernel_size=4, stride=2, padding=1)
x = torch.randn(20, 16, 32, 32)
output = conv_transpose2d_2x(x)

# H_out = (32-1)*2 - 2*1 + 1*(4-1) + 0 + 1 = 62 - 2 + 3 + 1 = 64
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Exact 2x upsampling: {output.shape[2] == 2 * x.shape[2] and output.shape[3] == 2 * x.shape[3]}")

Input shape: torch.Size([20, 16, 32, 32])
Output shape: torch.Size([20, 33, 64, 64])
Exact 2x upsampling: True


In [87]:
# Example 3: Non-square kernels and unequal stride with padding
conv_transpose2d_nonsq = nn.ConvTranspose2d(16, 33, kernel_size=(3, 5), stride=(2, 1), padding=(1, 2))
x = torch.randn(20, 16, 32, 64)
output = conv_transpose2d_nonsq(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 32, 64])
Output shape: torch.Size([20, 33, 63, 64])


In [88]:
# Example 4: Using output_padding to resolve ambiguity
# Two different input sizes (63 and 64) would produce the same output (32) with Conv2d(stride=2)
# output_padding helps specify we want 64, not 63

conv_transpose2d_op = nn.ConvTranspose2d(16, 33, kernel_size=3, stride=2, padding=1, output_padding=1)
x = torch.randn(20, 16, 32, 32)
output = conv_transpose2d_op(x)

# H_out = (32-1)*2 - 2*1 + 1*(3-1) + 1 + 1 = 62 - 2 + 2 + 1 + 1 = 64
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 32, 32])
Output shape: torch.Size([20, 33, 64, 64])


In [89]:
# Example 5: GAN-style generator upsampling chain
# Starting from a small spatial size and progressively upsampling

class SimpleGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        # Each layer doubles spatial dimensions: 4 -> 8 -> 16 -> 32 -> 64
        self.layers = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),  # 4 -> 8
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 8 -> 16
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # 16 -> 32
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),     # 32 -> 64
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.layers(x)

generator = SimpleGenerator()
latent = torch.randn(1, 512, 4, 4)  # Starting from 4x4 feature map
generated_image = generator(latent)

print(f"Latent shape: {latent.shape}")
print(f"Generated image shape: {generated_image.shape}")

Latent shape: torch.Size([1, 512, 4, 4])
Generated image shape: torch.Size([1, 3, 64, 64])


In [90]:
# Example 6: Comparison of Conv2d and ConvTranspose2d (inverse spatial transform)
H_original, W_original = 64, 64
x_original = torch.randn(1, 3, H_original, W_original)

# Downsample with Conv2d
conv2d_down = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
x_down = conv2d_down(x_original)

# Upsample with ConvTranspose2d (matching parameters)
conv_transpose2d_up = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1)
x_up = conv_transpose2d_up(x_down)

print(f"Original shape: {x_original.shape}")
print(f"After Conv2d (downsample): {x_down.shape}")
print(f"After ConvTranspose2d (upsample): {x_up.shape}")
print(f"Spatial dimensions restored: {x_original.shape[2:] == x_up.shape[2:]}")

Original shape: torch.Size([1, 3, 64, 64])
After Conv2d (downsample): torch.Size([1, 64, 32, 32])
After ConvTranspose2d (upsample): torch.Size([1, 3, 64, 64])
Spatial dimensions restored: True


In [92]:
# Verify the output size formula for ConvTranspose2d
def conv_transpose2d_output_size(H_in, W_in, kernel_size, stride=1, padding=0, dilation=1, output_padding=0):
    """Calculate ConvTranspose2d output dimensions"""
    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)
    if isinstance(stride, int):
        stride = (stride, stride)
    if isinstance(padding, int):
        padding = (padding, padding)
    if isinstance(dilation, int):
        dilation = (dilation, dilation)
    if isinstance(output_padding, int):
        output_padding = (output_padding, output_padding)
    
    H_out = (H_in - 1) * stride[0] - 2 * padding[0] + dilation[0] * (kernel_size[0] - 1) + output_padding[0] + 1
    W_out = (W_in - 1) * stride[1] - 2 * padding[1] + dilation[1] * (kernel_size[1] - 1) + output_padding[1] + 1
    return H_out, W_out

# Test with various configurations
H_in, W_in = 32, 32
x = torch.randn(1, 16, H_in, W_in)

configs = [
    {'kernel_size': 3, 'stride': 1, 'padding': 0},
    {'kernel_size': 4, 'stride': 2, 'padding': 1},
    {'kernel_size': 3, 'stride': 2, 'padding': 1, 'output_padding': 1},
    {'kernel_size': 3, 'stride': 1, 'padding': 0, 'dilation': 2},
]

for config in configs:
    conv_t = nn.ConvTranspose2d(16, 33, **config)
    output = conv_t(x)
    H_out_calc, W_out_calc = conv_transpose2d_output_size(H_in, W_in, **config)
    print(f"Config: {config}")
    print(f"  Calculated: ({H_out_calc}, {W_out_calc}), Actual: {tuple(output.shape[2:])}")
    print(f"  Match: {(H_out_calc, W_out_calc) == tuple(output.shape[2:])}")

Config: {'kernel_size': 3, 'stride': 1, 'padding': 0}
  Calculated: (34, 34), Actual: (34, 34)
  Match: True
Config: {'kernel_size': 4, 'stride': 2, 'padding': 1}
  Calculated: (64, 64), Actual: (64, 64)
  Match: True
Config: {'kernel_size': 3, 'stride': 2, 'padding': 1, 'output_padding': 1}
  Calculated: (64, 64), Actual: (64, 64)
  Match: True
Config: {'kernel_size': 3, 'stride': 1, 'padding': 0, 'dilation': 2}
  Calculated: (36, 36), Actual: (36, 36)
  Match: True


### nn.ConvTranspose3d

<span style="font-size: 15px;">

#### Where is it used?

The `nn.ConvTranspose3d` layer applies a 3D transposed convolution over an input signal composed of several input planes. It is used for:

- **Video generation**: Upsampling in video GANs, video prediction models
- **Medical image reconstruction**: Upsampling in 3D U-Net for CT/MRI segmentation
- **3D object generation**: Voxel-based generative models
- **Volumetric super-resolution**: Increasing resolution of 3D data

#### Input and Output Shapes

- **Input**: $(B, C_{\text{in}}, D_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ where:
  - $B$ = batch size
  - $C_{\text{in}}$ = number of input channels
  - $D_{\text{in}}$ = depth of the input (e.g., number of frames, slices)
  - $H_{\text{in}}$ = height of the input
  - $W_{\text{in}}$ = width of the input

- **Output**: $(B, C_{\text{out}}, D_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ where:
  - $C_{\text{out}}$ = number of output channels
  - $D_{\text{out}} = (D_{\text{in}} - 1) \times \text{stride}[0] - 2 \times \text{padding}[0] + \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) + \text{output\_padding}[0] + 1$
  - $H_{\text{out}} = (H_{\text{in}} - 1) \times \text{stride}[1] - 2 \times \text{padding}[1] + \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) + \text{output\_padding}[1] + 1$
  - $W_{\text{out}} = (W_{\text{in}} - 1) \times \text{stride}[2] - 2 \times \text{padding}[2] + \text{dilation}[2] \times (\text{kernel\_size}[2] - 1) + \text{output\_padding}[2] + 1$

#### Default Arguments
```python
nn.ConvTranspose3d(in_channels, out_channels, kernel_size, stride=1, padding=0,
                   output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
```

- `in_channels` (int): Number of channels in the input volume (required)
- `out_channels` (int): Number of channels produced by the convolution (required)
- `kernel_size` (int or tuple): Size of the convolving kernel (required)
- `stride` (int or tuple): Stride of the convolution (controls upsampling factor). Default: `1`
- `padding` (int or tuple): Zero-padding added to all sides. **Reduces** output size. Default: `0`
- `output_padding` (int or tuple): Additional size added to one side. Must be `< stride`. Default: `0`
- `dilation` (int or tuple): Spacing between kernel elements. Default: `1`
- `groups` (int): Number of blocked connections. Default: `1`
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- `padding_mode` (str): Only `'zeros'` is supported. Default: `'zeros'`

#### Weight Shape

The weight tensor has shape: $(C_{\text{in}}, \frac{C_{\text{out}}}{\text{groups}}, \text{kernel\_size}[0], \text{kernel\_size}[1], \text{kernel\_size}[2])$

**Note**: The weight shape is **transposed** compared to `nn.Conv3d` which has shape $(C_{\text{out}}, \frac{C_{\text{in}}}{\text{groups}}, K_d, K_h, K_w)$.

#### Mathematical Formulation

For an input of size $(B, C_{\text{in}}, D_{\text{in}}, H_{\text{in}}, W_{\text{in}})$ and output of size $(B, C_{\text{out}}, D_{\text{out}}, H_{\text{out}}, W_{\text{out}})$, the transposed convolution from $x_{\rm in}[b, c, d, h, w]$ to $y_{\rm out}[b, c, d', h', w']$ is performed as follows:

* For `groups`$ = 1$:
$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, d', h', w'] = &\sum_{c_{\rm in}=0}^{C_{\rm in}-1} \sum_{k_d=0}^{K_d-1} \sum_{k_h=0}^{K_h-1} \sum_{k_w=0}^{K_w-1} \\
&\tilde{x}_{\rm in}[b, c_{\rm in}, d' + p_d - d_d k_d, h' + p_h - d_h k_h, w' + p_w - d_w k_w] \\
&\cdot \omega[c_{\rm in}, c_{\rm out}, K_d - 1 - k_d, K_h - 1 - k_h, K_w - 1 - k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

where $\tilde{x}_{\rm in}$ is the input after zero-insertion (inserting $s_d - 1$, $s_h - 1$, and $s_w - 1$ zeros between elements along depth, height, and width respectively).

Equivalently, as a **summation over input positions**:

$$
\begin{aligned}
y_{\rm out}[b, c_{\rm out}, d', h', w'] = &\sum_{c_{\rm in}=0}^{C_{\rm in}-1} \sum_{d=0}^{D_{\rm in}-1} \sum_{h=0}^{H_{\rm in}-1} \sum_{w=0}^{W_{\rm in}-1} \sum_{k_d=0}^{K_d-1} \sum_{k_h=0}^{K_h-1} \sum_{k_w=0}^{K_w-1} \\
&\mathbf{1}_{\substack{[d' = s_d d + d_d k_d - p_d + p_{\rm out,d}] \\ [h' = s_h h + d_h k_h - p_h + p_{\rm out,h}] \\ [w' = s_w w + d_w k_w - p_w + p_{\rm out,w}]}} \cdot x_{\rm in}[b, c_{\rm in}, d, h, w] \cdot \omega[c_{\rm in}, c_{\rm out}, k_d, k_h, k_w] + \beta[c_{\rm out}]
\end{aligned}
$$

where $(s_d, s_h, s_w) = $ `stride`, $(d_d, d_h, d_w) = $ `dilation`, $(p_d, p_h, p_w) = $ `padding`, $(p_{\rm out,d}, p_{\rm out,h}, p_{\rm out,w}) = $ `output_padding`, $(K_d, K_h, K_w) = $ `kernel_size`, and $\mathbf{1}_{[\cdot]}$ is the indicator function.

* For `groups`$ > 1$, the channels are split into groups analogously to regular convolutions.

**Remarks**:
1. Each input voxel "scatters" its influence to a $K_d \times K_h \times K_w$ region in the output.
2. If `kernel_size = k` is just a number, then PyTorch automatically assumes $(K_d, K_h, K_w) = (k, k, k)$.

</span>

In [93]:
# Basic nn.ConvTranspose3d usage
# Input: 16 channels, Output: 33 channels, kernel size: 3x3x3
conv_transpose3d = nn.ConvTranspose3d(in_channels=16, out_channels=33, kernel_size=3)

print(f"Weight shape: {conv_transpose3d.weight.shape}")  # (in_channels, out_channels, kD, kH, kW)
print(f"Bias shape: {conv_transpose3d.bias.shape}")      # (out_channels,)

Weight shape: torch.Size([16, 33, 3, 3, 3])
Bias shape: torch.Size([33])


In [94]:
# Example 1: Basic transposed convolution (slight upsampling)
conv_transpose3d = nn.ConvTranspose3d(16, 33, kernel_size=3)
x = torch.randn(4, 16, 10, 50, 100)  # (batch, channels, depth, height, width)
output = conv_transpose3d(x)

# D_out = (10-1)*1 - 2*0 + 1*(3-1) + 0 + 1 = 12
# H_out = (50-1)*1 - 2*0 + 1*(3-1) + 0 + 1 = 52
# W_out = (100-1)*1 - 2*0 + 1*(3-1) + 0 + 1 = 102
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([4, 16, 10, 50, 100])
Output shape: torch.Size([4, 33, 12, 52, 102])


In [95]:
# Example 2: 2x upsampling in all dimensions
conv_transpose3d_2x = nn.ConvTranspose3d(16, 33, kernel_size=4, stride=2, padding=1)
x = torch.randn(4, 16, 8, 16, 16)
output = conv_transpose3d_2x(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Exact 2x upsampling in all dims: {all(o == 2*i for o, i in zip(output.shape[2:], x.shape[2:]))}")

Input shape: torch.Size([4, 16, 8, 16, 16])
Output shape: torch.Size([4, 33, 16, 32, 32])
Exact 2x upsampling in all dims: True


In [96]:
# Example 3: Non-cubic kernels and unequal stride (e.g., upsample only spatially, not temporally)
# kernel_size=(depth, height, width), stride=(sD, sH, sW), padding=(pD, pH, pW)
conv_transpose3d_spatial = nn.ConvTranspose3d(16, 33, kernel_size=(3, 4, 4), stride=(1, 2, 2), padding=(1, 1, 1))
x = torch.randn(4, 16, 16, 32, 32)  # Video: 16 frames, 32x32 resolution
output = conv_transpose3d_spatial(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Temporal dim preserved: {x.shape[2] == output.shape[2]}")
print(f"Spatial dims doubled: {output.shape[3] == 2*x.shape[3] and output.shape[4] == 2*x.shape[4]}")

Input shape: torch.Size([4, 16, 16, 32, 32])
Output shape: torch.Size([4, 33, 16, 64, 64])
Temporal dim preserved: True
Spatial dims doubled: True


In [97]:
# Example 4: Medical imaging - 3D U-Net decoder block
# Upsampling feature maps in a 3D segmentation network

class UNet3DDecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.upsample = nn.ConvTranspose3d(in_channels, out_channels, 
                                            kernel_size=2, stride=2)  # 2x upsample
        self.conv = nn.Conv3d(out_channels * 2, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x, skip_connection):
        x = self.upsample(x)
        x = torch.cat([x, skip_connection], dim=1)  # Skip connection from encoder
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

# Simulate decoder block usage
decoder_block = UNet3DDecoderBlock(128, 64)
bottleneck_features = torch.randn(1, 128, 8, 8, 8)  # From encoder bottleneck
skip_features = torch.randn(1, 64, 16, 16, 16)      # Skip connection from encoder
output = decoder_block(bottleneck_features, skip_features)

print(f"Bottleneck features: {bottleneck_features.shape}")
print(f"Skip connection: {skip_features.shape}")
print(f"Decoder output: {output.shape}")

Bottleneck features: torch.Size([1, 128, 8, 8, 8])
Skip connection: torch.Size([1, 64, 16, 16, 16])
Decoder output: torch.Size([1, 64, 16, 16, 16])


In [98]:
# Example 5: Video generation - upsampling from latent to video frames
# Starting from a compressed spatiotemporal representation

class VideoGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        # Progressively upsample: (2,4,4) -> (4,8,8) -> (8,16,16) -> (16,32,32)
        self.layers = nn.Sequential(
            nn.ConvTranspose3d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.ConvTranspose3d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.layers(x)

generator = VideoGenerator()
latent = torch.randn(1, 256, 2, 4, 4)  # Compressed representation
video = generator(latent)

print(f"Latent shape: {latent.shape}")
print(f"Generated video shape: {video.shape}")
print(f"(batch, RGB channels, frames, height, width)")

Latent shape: torch.Size([1, 256, 2, 4, 4])
Generated video shape: torch.Size([1, 3, 16, 32, 32])
(batch, RGB channels, frames, height, width)


In [99]:
# Verify the output size formula for ConvTranspose3d
def conv_transpose3d_output_size(D_in, H_in, W_in, kernel_size, stride=1, padding=0, dilation=1, output_padding=0):
    """Calculate ConvTranspose3d output dimensions"""
    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size, kernel_size)
    if isinstance(stride, int):
        stride = (stride, stride, stride)
    if isinstance(padding, int):
        padding = (padding, padding, padding)
    if isinstance(dilation, int):
        dilation = (dilation, dilation, dilation)
    if isinstance(output_padding, int):
        output_padding = (output_padding, output_padding, output_padding)
    
    D_out = (D_in - 1) * stride[0] - 2 * padding[0] + dilation[0] * (kernel_size[0] - 1) + output_padding[0] + 1
    H_out = (H_in - 1) * stride[1] - 2 * padding[1] + dilation[1] * (kernel_size[1] - 1) + output_padding[1] + 1
    W_out = (W_in - 1) * stride[2] - 2 * padding[2] + dilation[2] * (kernel_size[2] - 1) + output_padding[2] + 1
    return D_out, H_out, W_out

# Test
D_in, H_in, W_in = 8, 16, 16
x = torch.randn(1, 16, D_in, H_in, W_in)

configs = [
    {'kernel_size': 3, 'stride': 1, 'padding': 0},
    {'kernel_size': 4, 'stride': 2, 'padding': 1},
    {'kernel_size': (3, 4, 4), 'stride': (1, 2, 2), 'padding': (1, 1, 1)},
    {'kernel_size': 2, 'stride': 2, 'padding': 0},
]

for config in configs:
    conv_t = nn.ConvTranspose3d(16, 33, **config)
    output = conv_t(x)
    D_out_calc, H_out_calc, W_out_calc = conv_transpose3d_output_size(D_in, H_in, W_in, **config)
    print(f"Config: {config}")
    print(f"  Calculated: ({D_out_calc}, {H_out_calc}, {W_out_calc}), Actual: {tuple(output.shape[2:])}")
    print(f"  Match: {(D_out_calc, H_out_calc, W_out_calc) == tuple(output.shape[2:])}")
    print()

Config: {'kernel_size': 3, 'stride': 1, 'padding': 0}
  Calculated: (10, 18, 18), Actual: (10, 18, 18)
  Match: True

Config: {'kernel_size': 4, 'stride': 2, 'padding': 1}
  Calculated: (16, 32, 32), Actual: (16, 32, 32)
  Match: True

Config: {'kernel_size': (3, 4, 4), 'stride': (1, 2, 2), 'padding': (1, 1, 1)}
  Calculated: (8, 32, 32), Actual: (8, 32, 32)
  Match: True

Config: {'kernel_size': 2, 'stride': 2, 'padding': 0}
  Calculated: (16, 32, 32), Actual: (16, 32, 32)
  Match: True



### Summary

#### Note on the Mathematical Formulation of Transposed Convolutions

Throughout this notebook, we presented two equivalent formulations for transposed convolutions:

**Formulation 1 (Flipped Kernel View):**
$$
y_{\rm out}[\ldots, j] = \sum_{c_{\rm in}, k} \tilde{x}_{\rm in}[\ldots, j + p - d \cdot k] \cdot \omega[c_{\rm in}, c_{\rm out}, K - 1 - k] + \beta[c_{\rm out}]
$$

**Formulation 2 (Scatter/Adjoint View):**
$$
y_{\rm out}[\ldots, j] = \sum_{c_{\rm in}, i, k} \mathbf{1}_{[j = s \cdot i + d \cdot k - p + p_{\rm out}]} \cdot x_{\rm in}[\ldots, i] \cdot \omega[c_{\rm in}, c_{\rm out}, k] + \beta[c_{\rm out}]
$$

These formulations are **mathematically equivalent** via the change of variables $k' = K - 1 - k$. However, they represent different conceptual interpretations:

- **Flipped Kernel View**: The transposed convolution can be understood as inserting zeros between input elements, then applying a regular convolution with a spatially flipped kernel. This interpretation arises naturally when deriving the transposed convolution as the gradient of a forward convolution.

- **Scatter/Adjoint View**: Each input element "scatters" or "broadcasts" its value to multiple output positions, weighted by the kernel elements. This is the **interpretation that matches PyTorch's actual implementation** — the kernel weights are used directly without spatial flipping.

Lets verify this behavior:

In [105]:
# Create a ConvTranspose1d with a known kernel
conv_t = nn.ConvTranspose1d(1, 1, kernel_size=3, bias=False)
conv_t.weight.data = torch.tensor([[[1., 2., 3.]]])  # kernel = [1, 2, 3]

# Single input
x = torch.tensor([[[1.]]])  # single value at position 0
output = conv_t(x)
print(output)  # tensor([[[1., 2., 3.]]])  — NOT [3, 2, 1]

tensor([[[1., 2., 3.]]], grad_fn=<ConvolutionBackward0>)


In [100]:
# Summary comparison of all transposed convolutional layers
print("=" * 80)
print("TRANSPOSED CONVOLUTIONAL LAYER COMPARISON SUMMARY")
print("=" * 80)

# nn.ConvTranspose1d
conv_t1d = nn.ConvTranspose1d(64, 128, kernel_size=4, stride=2, padding=1)
x_1d = torch.randn(32, 64, 50)
print(f"\nnn.ConvTranspose1d(64, 128, kernel_size=4, stride=2, padding=1):")
print(f"  Input shape:  {tuple(x_1d.shape)} -> (batch, in_channels, length)")
print(f"  Output shape: {tuple(conv_t1d(x_1d).shape)} -> (batch, out_channels, length_upsampled)")
print(f"  Weight shape: {tuple(conv_t1d.weight.shape)} -> (in_ch, out_ch, kernel)")
print(f"  Parameters:   {sum(p.numel() for p in conv_t1d.parameters()):,}")

# nn.ConvTranspose2d
conv_t2d = nn.ConvTranspose2d(64, 128, kernel_size=4, stride=2, padding=1)
x_2d = torch.randn(32, 64, 28, 28)
print(f"\nnn.ConvTranspose2d(64, 128, kernel_size=4, stride=2, padding=1):")
print(f"  Input shape:  {tuple(x_2d.shape)} -> (batch, in_channels, H, W)")
print(f"  Output shape: {tuple(conv_t2d(x_2d).shape)} -> (batch, out_channels, H_up, W_up)")
print(f"  Weight shape: {tuple(conv_t2d.weight.shape)} -> (in_ch, out_ch, kH, kW)")
print(f"  Parameters:   {sum(p.numel() for p in conv_t2d.parameters()):,}")

# nn.ConvTranspose3d
conv_t3d = nn.ConvTranspose3d(64, 128, kernel_size=4, stride=2, padding=1)
x_3d = torch.randn(4, 64, 8, 14, 14)
print(f"\nnn.ConvTranspose3d(64, 128, kernel_size=4, stride=2, padding=1):")
print(f"  Input shape:  {tuple(x_3d.shape)} -> (batch, in_channels, D, H, W)")
print(f"  Output shape: {tuple(conv_t3d(x_3d).shape)} -> (batch, out_channels, D_up, H_up, W_up)")
print(f"  Weight shape: {tuple(conv_t3d.weight.shape)} -> (in_ch, out_ch, kD, kH, kW)")
print(f"  Parameters:   {sum(p.numel() for p in conv_t3d.parameters()):,}")

print("\n" + "=" * 80)

TRANSPOSED CONVOLUTIONAL LAYER COMPARISON SUMMARY

nn.ConvTranspose1d(64, 128, kernel_size=4, stride=2, padding=1):
  Input shape:  (32, 64, 50) -> (batch, in_channels, length)
  Output shape: (32, 128, 100) -> (batch, out_channels, length_upsampled)
  Weight shape: (64, 128, 4) -> (in_ch, out_ch, kernel)
  Parameters:   32,896

nn.ConvTranspose2d(64, 128, kernel_size=4, stride=2, padding=1):
  Input shape:  (32, 64, 28, 28) -> (batch, in_channels, H, W)
  Output shape: (32, 128, 56, 56) -> (batch, out_channels, H_up, W_up)
  Weight shape: (64, 128, 4, 4) -> (in_ch, out_ch, kH, kW)
  Parameters:   131,200

nn.ConvTranspose3d(64, 128, kernel_size=4, stride=2, padding=1):
  Input shape:  (4, 64, 8, 14, 14) -> (batch, in_channels, D, H, W)
  Output shape: (4, 128, 16, 28, 28) -> (batch, out_channels, D_up, H_up, W_up)
  Weight shape: (64, 128, 4, 4, 4) -> (in_ch, out_ch, kD, kH, kW)
  Parameters:   524,416



In [101]:
# Key formulas for output size calculation
print("KEY FORMULAS FOR OUTPUT SIZE CALCULATION")
print("="*60)
print()
print("nn.ConvTranspose1d/ConvTranspose2d/ConvTranspose3d:")
print("  For each spatial dimension:")
print("  L_out = (L_in - 1) * stride - 2*padding + dilation*(kernel_size-1) + output_padding + 1")
print()
print("Common recipes for exact 2x upsampling:")
print("  - kernel_size=4, stride=2, padding=1 -> exact doubling")
print("  - kernel_size=2, stride=2, padding=0 -> exact doubling")
print("  - kernel_size=3, stride=2, padding=1, output_padding=1 -> exact doubling")
print()
print("Key differences from regular Conv:")
print("  - stride > 1 INCREASES output size (upsampling)")
print("  - padding DECREASES output size")
print("  - Weight shape is (in_ch, out_ch, ...) not (out_ch, in_ch, ...)")

KEY FORMULAS FOR OUTPUT SIZE CALCULATION

nn.ConvTranspose1d/ConvTranspose2d/ConvTranspose3d:
  For each spatial dimension:
  L_out = (L_in - 1) * stride - 2*padding + dilation*(kernel_size-1) + output_padding + 1

Common recipes for exact 2x upsampling:
  - kernel_size=4, stride=2, padding=1 -> exact doubling
  - kernel_size=2, stride=2, padding=0 -> exact doubling
  - kernel_size=3, stride=2, padding=1, output_padding=1 -> exact doubling

Key differences from regular Conv:
  - stride > 1 INCREASES output size (upsampling)
  - padding DECREASES output size
  - Weight shape is (in_ch, out_ch, ...) not (out_ch, in_ch, ...)


In [102]:
# Comparison: Conv vs ConvTranspose (inverse spatial transformations)
print("CONV vs CONVTRANSPOSE: INVERSE SPATIAL TRANSFORMATIONS")
print("="*60)
print()

# 1D example
print("1D Example:")
x1d = torch.randn(1, 16, 100)
conv1d = nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1)
conv_t1d = nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1)
print(f"  Original: {tuple(x1d.shape)}")
print(f"  After Conv1d(stride=2): {tuple(conv1d(x1d).shape)}")
print(f"  After ConvTranspose1d(stride=2): {tuple(conv_t1d(conv1d(x1d)).shape)}")
print()

# 2D example
print("2D Example:")
x2d = torch.randn(1, 3, 64, 64)
conv2d = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
conv_t2d = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1)
print(f"  Original: {tuple(x2d.shape)}")
print(f"  After Conv2d(stride=2): {tuple(conv2d(x2d).shape)}")
print(f"  After ConvTranspose2d(stride=2): {tuple(conv_t2d(conv2d(x2d)).shape)}")
print()

# 3D example
print("3D Example:")
x3d = torch.randn(1, 3, 16, 32, 32)
conv3d = nn.Conv3d(3, 64, kernel_size=4, stride=2, padding=1)
conv_t3d = nn.ConvTranspose3d(64, 3, kernel_size=4, stride=2, padding=1)
print(f"  Original: {tuple(x3d.shape)}")
print(f"  After Conv3d(stride=2): {tuple(conv3d(x3d).shape)}")
print(f"  After ConvTranspose3d(stride=2): {tuple(conv_t3d(conv3d(x3d)).shape)}")

CONV vs CONVTRANSPOSE: INVERSE SPATIAL TRANSFORMATIONS

1D Example:
  Original: (1, 16, 100)
  After Conv1d(stride=2): (1, 32, 50)
  After ConvTranspose1d(stride=2): (1, 16, 100)

2D Example:
  Original: (1, 3, 64, 64)
  After Conv2d(stride=2): (1, 64, 32, 32)
  After ConvTranspose2d(stride=2): (1, 3, 64, 64)

3D Example:
  Original: (1, 3, 16, 32, 32)
  After Conv3d(stride=2): (1, 64, 8, 16, 16)
  After ConvTranspose3d(stride=2): (1, 3, 16, 32, 32)


tensor([[[1., 2., 3.]]], grad_fn=<ConvolutionBackward0>)
